In [12]:
import pandas as pd
import re
from nltk.corpus import stopwords
import nltk.data

In [13]:
# Read data from files 
train = pd.read_csv( "train@urlpandaslower.csv", header=0, 
 delimiter=",", quoting=0 )
test = pd.read_csv( "test@urlpandaslower.csv", header=0, delimiter=",", quoting=0 )



In [14]:
# Verify the number of reviews that were read (100,000 in total)
print "Read %d labeled train messages, %d labeled test messages\n " \
    % (train["Message"].size,  test["Message"].size )

Read 1600000 labeled train messages, 498 labeled test messages
 


In [15]:
def message_to_wordlist( message, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    #review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    message_text = re.sub("[^a-zA-Z]"," ", message)
    #
    # 3. Convert words to lower case and split them
    words = message_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [16]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def message_to_sentences( message, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(message.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( message_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [18]:
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for message in train["Message"]:
    sentences += message_to_sentences(message, tokenizer)

#print "Parsing sentences from unlabeled set"
#for message in unlabeled_train["Message"]:
#    sentences += message_to_sentences(message, tokenizer)


Parsing sentences from training set


In [19]:
print len(sentences)


2701164


In [ ]:
print sentences[0]

In [21]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 400    # Word vector dimensionality                      
min_word_count = 50   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)


Training model...


In [22]:
model.doesnt_match("man woman child kitchen".split())

'kitchen'

In [23]:
model.doesnt_match("france england germany berlin".split())
'berlin'

'berlin'

In [24]:
model.doesnt_match("paris berlin london austria".split())
'paris'

'paris'

In [25]:
model.most_similar("awful")

[('unpleasant', 0.4942152500152588),
 ('horrible', 0.4806709885597229),
 ('understatement', 0.47916990518569946),
 ('dreadful', 0.4743342399597168),
 ('horrid', 0.470512717962265),
 ('terrible', 0.46351492404937744),
 ('shocking', 0.45535731315612793),
 ('accomplishment', 0.45230770111083984),
 ('intriguing', 0.44709300994873047),
 ('positively', 0.44331568479537964)]

In [26]:
# Load the model that we created in Part 2
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")

In [27]:
type(model.syn0)

numpy.ndarray

In [28]:
model.syn0.shape

(12648, 400)

In [29]:
#From Words To Paragraphs, Attempt 1: Vector Averaging
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(messages, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    messageFeatureVecs = np.zeros((len(messages),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in messages:
        #
        # Print a status message every 1000th review
       if counter%1000. == 0.:
           print "Message %d of %d" % (counter, len(messages))
        # 
        # Call the function (defined above) that makes average feature vectors
       messageFeatureVecs[counter] = makeFeatureVec(message, model, \
           num_features)
       #
        # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs

In [31]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_messages = []
for message in train["Message"]:
    clean_train_messages.append( message_to_wordlist( message, \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_messages, model, num_features )

print "Creating average feature vecs for test messages"
clean_test_messages = []
for message in test["Message"]:
    clean_test_messages.append( message_to_wordlist( message, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_messages, model, num_features )

Review 0 of 1600000
Review 1000 of 1600000
Review 2000 of 1600000
Review 3000 of 1600000
Review 4000 of 1600000
Review 5000 of 1600000
Review 6000 of 1600000
Review 7000 of 1600000
Review 8000 of 1600000
Review 9000 of 1600000
Review 10000 of 1600000
Review 11000 of 1600000
Review 12000 of 1600000
Review 13000 of 1600000
Review 14000 of 1600000
Review 15000 of 1600000
Review 16000 of 1600000
Review 17000 of 1600000
Review 18000 of 1600000
Review 19000 of 1600000
Review 20000 of 1600000
Review 21000 of 1600000
Review 22000 of 1600000
Review 23000 of 1600000
Review 24000 of 1600000
Review 25000 of 1600000
Review 26000 of 1600000
Review 27000 of 1600000
Review 28000 of 1600000
Review 29000 of 1600000
Review 30000 of 1600000
Review 31000 of 1600000
Review 32000 of 1600000
Review 33000 of 1600000
Review 34000 of 1600000
Review 35000 of 1600000
Review 36000 of 1600000
Review 37000 of 1600000
Review 38000 of 1600000
Review 39000 of 1600000
Review 40000 of 1600000
Review 41000 of 1600000
Revie

In [ ]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print "Fitting a random forest to labeled training data..."
forest = forest.fit( trainDataVecs, train["Sentiment"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"Message":test["Message"], "Sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=0 )